**Import libraries**

In [1]:
import os
os.chdir("/exp/sbnd/app/users/svidales/AI_nuvT")

In [2]:
from imports import *

2025-05-28 19:50:52.319726: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-28 19:50:52.319835: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-28 19:50:52.321723: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-28 19:50:52.327539: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1. Preprocessing

The data corresponds to a MC simulation of the SBND experiment used in the paper "Scintillation Light in SBND: Simulation, Reconstruction, and Expected Performance of the Photon Detection System" in https://arxiv.org/abs/2406.07514. It is a simulation of BNB + Cosmics and their subsequent interaction in SBND, as well as the simulation of the detector's response to the particles resulting from the interaction of the neutrinos.

### Load variables

**archivos en espacio personal - a partir de ahora correrlos en data**

In [3]:
file_path = '/exp/sbnd/data/users/svidales/opana_tree_combined_v2304_complete.root'
file = uproot.open(file_path)
optree = file['opanatree']['OpAnaTree'] # Tree con número de fotoelectrones
print("Keys in optree:", optree.keys())

Keys in optree: ['eventID', 'runID', 'subrunID', 'nuvX', 'nuvY', 'nuvZ', 'nuvT', 'nuvE', 'stepX', 'stepY', 'stepZ', 'stepT', 'dE', 'energydep', 'energydepX', 'energydepY', 'energydepZ', 'E', 'StartPx', 'StartPy', 'StartPz', 'EndPx', 'EndPy', 'EndPz', 'process', 'trackID', 'motherID', 'PDGcode', 'InTimeCosmics', 'InTimeCosmicsTime', 'dEtpc', 'dEpromx', 'dEpromy', 'dEpromz', 'dEspreadx', 'dEspready', 'dEspreadz', 'dElowedges', 'dEmaxedges', 'nopflash', 'flash_id', 'flash_time', 'flash_total_pe', 'flash_pe_v', 'flash_tpc', 'flash_y', 'flash_yerr', 'flash_z', 'flash_zerr', 'flash_x', 'flash_xerr', 'flash_ophit_time', 'flash_ophit_risetime', 'flash_ophit_starttime', 'flash_ophit_amp', 'flash_ophit_area', 'flash_ophit_width', 'flash_ophit_pe', 'flash_ophit_ch']


In [4]:
# Input variables
# f_ophit_PE: number of photoelectrons (PEs) per OpHit
# f_ophit_ch: number of channel that collect the OpHit
# f_ophit_t: OpHit time

# Labels
# nuvT: neutrino interaction time
# dEprom{x,y,z}: energy barycenter coordinates {x,y,z}

# Auxiliary variables
# nuvZ: z-coordinate of the neutrino interaction
# dEtpc: allows filtering by energy deposited in the TPC

# These are awkward arrays (i.e., irregular structures) with a 3-level hierarchy (events -> flashes -> hits)

f_ophit_PE, f_ophit_ch, f_ophit_t, nuvT, dEpromx, dEpromy, dEpromz, dEtpc, nuvZ = (
    optree[key].array() for key in 
    ['flash_ophit_pe', 'flash_ophit_ch', 'flash_ophit_time', 'nuvT', 'dEpromx', 'dEpromy', 'dEpromz', 'dEtpc', "nuvZ"]
)

In [5]:
print(len(f_ophit_PE))

243599


In [27]:
print(nuvT[5])

[1.41e+03, 994, 1.55e+03, 456]


In [28]:
print(dEpromx[5])

[-999, 75.5]


In [11]:
print(f_ophit_t[0][1])

[8.13, 7.97, 7.9, 8.03, 7.68, 7.58, ..., 0.448, 0.416, 0.416, 0.416, 0.416]


In [7]:
from collections import Counter
lengths = [len(inner) for inner in nuvT]
count = Counter(lengths)

# Get counts for 1 through 4
result = {i: count.get(i, 0) for i in range(1, 5)}
print(result)

{1: 102618, 2: 96137, 3: 35365, 4: 7867}


**Eliminate events with more than one neutrino & events with no flashes**

In [6]:
# Filter events where nuvT has exactly one element
mask = ak.num(nuvT) == 1  

# Apply the mask to all variables
f_ophit_PE_1, f_ophit_ch_1, f_ophit_t_1 = f_ophit_PE[mask], f_ophit_ch[mask], f_ophit_t[mask]
nuvT_1, dEpromx_1, dEpromy_1, dEpromz_1, dEtpc_1, nuvZ_1 = nuvT[mask], dEpromx[mask], dEpromy[mask], dEpromz[mask], dEtpc[mask], nuvZ[mask]

In [7]:
print(len(f_ophit_PE_1))

102618


In [9]:
del f_ophit_PE, f_ophit_ch, f_ophit_t, nuvT, dEpromx, dEpromy, dEpromz, dEtpc, nuvZ

In [10]:
# Filter events with at least one flash
mask = ak.num(f_ophit_PE_1, axis=1) > 0  

# Apply the mask to all variables
f_ophit_PE_2, f_ophit_ch_2, f_ophit_t_2 = f_ophit_PE_1[mask], f_ophit_ch_1[mask], f_ophit_t_1[mask]
nuvT_2, dEpromx_2, dEpromy_2, dEpromz_2, dEtpc_2, nuvZ_2 = nuvT_1[mask], dEpromx_1[mask], dEpromy_1[mask], dEpromz_1[mask], dEtpc_1[mask], nuvZ_1[mask]


In [11]:
print(len(f_ophit_PE_2))

101189


In [12]:
del f_ophit_PE_1, f_ophit_ch_1, f_ophit_t_1, nuvT_1, dEpromx_1, dEpromy_1, dEpromz_1, dEtpc_1, nuvZ_1

### Corrección PTM delay

**Correction PMT delay 135 ns due to the difference between the photon arrival times (at the photocathode)
and the digitised signal (at the anode)**

**es posible que luego añada las coordenadas en la siguiente celda y lo guarde**

In [13]:
PDSMap = file['opanatree']['PDSMapTree']
ID = PDSMap['OpDetID'].array()
Type = PDSMap['OpDetType'].array()
channel_dict = {id_val: (int(type_val)) for id_val, type_val in zip(ID[0],Type[0])}
print(channel_dict)

{0: 3, 1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 2, 19: 2, 20: 2, 21: 2, 22: 2, 23: 2, 24: 3, 25: 3, 26: 3, 27: 3, 28: 3, 29: 3, 30: 3, 31: 3, 32: 3, 33: 3, 34: 3, 35: 3, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 2, 43: 2, 44: 2, 45: 2, 46: 2, 47: 2, 48: 2, 49: 2, 50: 2, 51: 2, 52: 2, 53: 2, 54: 3, 55: 3, 56: 3, 57: 3, 58: 3, 59: 3, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 2, 73: 2, 74: 2, 75: 2, 76: 2, 77: 2, 78: 3, 79: 3, 80: 3, 81: 3, 82: 3, 83: 3, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 2, 97: 2, 98: 2, 99: 2, 100: 2, 101: 2, 102: 3, 103: 3, 104: 3, 105: 3, 106: 3, 107: 3, 108: 3, 109: 3, 110: 3, 111: 3, 112: 3, 113: 3, 114: 1, 115: 1, 116: 1, 117: 1, 118: 1, 119: 1, 120: 2, 121: 2, 122: 2, 123: 2, 124: 2, 125: 2, 126: 2, 127: 2, 128: 2, 129: 2, 130: 2, 131: 2, 132: 3, 133: 3, 134: 3, 135: 3, 136: 3, 137: 3, 138: 

In [14]:
# Create the list of channels to correct
channels_to_correct = [ch for ch, value in channel_dict.items() if value in {0, 1}]
print(channels_to_correct)

# Create a mask for the channels to correct
mask = ak.Array([
    [[ch in channels_to_correct for ch in ophit] for ophit in flash] for flash in f_ophit_ch_2
])

# Apply the mask to f_ophit_t variable
f_ophit_t_adj = ak.where(mask, f_ophit_t_2 - 0.135, f_ophit_t_2)

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 36, 37, 38, 39, 40, 41, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 114, 115, 116, 117, 118, 119, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 192, 193, 194, 195, 196, 197, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 270, 271, 272, 273, 274, 275, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305]


In [14]:
del f_ophit_t_2

### Selección de TPC para la variable dEprom{x,y,z} y (Opcional) para los flashes de las variables flash_ophit

In [15]:
import awkward as ak
import numpy as np

# --- 1. Clasificación de canales ---
pmt_channels = [ch for ch, val in channel_dict.items() if val in {0, 1}]
xas_channels = [ch for ch, val in channel_dict.items() if val in {2, 3}]

def split_even_odd(channels):
    return set(filter(lambda x: x % 2 == 0, channels)), set(filter(lambda x: x % 2 != 0, channels))

pmt_even, pmt_odd = split_even_odd(pmt_channels)
xas_even, xas_odd = split_even_odd(xas_channels)

def categorize_first_ch(vector):
    if isinstance(vector, (list, ak.Array)) and len(vector) > 0:
        ch = vector[0]
        if ch in pmt_even: return 0
        if ch in pmt_odd:  return 1
        if ch in xas_even: return 2
        if ch in xas_odd:  return 3
    return -1  # No clasificado o vacío

categorized_flashes = ak.Array([
    [categorize_first_ch(flash) for flash in event]
    for event in ak.to_list(f_ophit_ch_2)
])

# --- 2. Sumar los PEs de cada flash ---
sum_pe = ak.sum(f_ophit_PE_2, axis=2)  # [evento][flash]

# --- 3. Crear máscaras por categoría ---
mask_even = (categorized_flashes == 0) | (categorized_flashes == 2)
mask_odd  = (categorized_flashes == 1) | (categorized_flashes == 3)

# --- 4. Sumar PE por grupo ---
sum_even = ak.sum(ak.where(mask_even, sum_pe, 0), axis=1)
sum_odd  = ak.sum(ak.where(mask_odd, sum_pe, 0), axis=1)

# --- 5. Selección del grupo con mayor PE si hay más de 2 flashes ---
n_flashes = ak.num(categorized_flashes)
decision = sum_even >= sum_odd 

# Generar máscara de selección
selected_mask = ak.Array([
    np.ones(n, dtype=bool) if n <= 2  # Keep all flashes for ≤ 2
    else (mask_even[i] if decision[i] else mask_odd[i])
    for i, n in enumerate(ak.to_list(n_flashes))
])

# --- 6. Función para filtrar un array usando la máscara ---
def filter_by_mask(array, mask):
    return ak.Array([
        [item for item, flag in zip(event, event_mask) if flag]
        for event, event_mask in zip(ak.to_list(array), ak.to_list(mask))
    ])

# --- 7. Aplicar máscaras de selección ---
f_ophit_t_adj_sel       = filter_by_mask(f_ophit_t_adj, selected_mask)
f_ophit_PE_2_sel        = filter_by_mask(f_ophit_PE_2, selected_mask)
f_ophit_ch_2_sel        = filter_by_mask(f_ophit_ch_2, selected_mask)
categorized_flashes_sel = filter_by_mask(categorized_flashes, selected_mask)

# --- 8. Selección TPC ganadora ---
selector = ak.Array([[d, not d] for d in decision])

def select_tpc_value(array_2d, selector):
    return ak.sum(ak.where(selector, array_2d, 0), axis=1)

dEpromx_sel = select_tpc_value(dEpromx_2, selector)
dEpromy_sel = select_tpc_value(dEpromy_2, selector)
dEpromz_sel = select_tpc_value(dEpromz_2, selector)
dEtpc_sel   = select_tpc_value(dEtpc_2, selector)

In [16]:
del f_ophit_t_adj, f_ophit_PE_2, f_ophit_ch_2, categorized_flashes, dEpromx_2, dEpromy_2, dEpromz_2, dEtpc_2, decision, mask_even, mask_odd, sum_even, sum_odd, n_flashes

### Para reconstrucción temporal

In [16]:
# Create a boolean mask where dEpromx_f_unique is not -999 & select events with deposition >50 MeV (dEtpc_f > 50)

mask = (dEpromx_sel != -999) & (dEpromy_sel != -999) & (dEpromz_sel != -999) & (dEtpc_sel > 50)
mask_1d = ak.to_numpy(mask)

# Apply the mask to both the image and dEpromx_f_unique to keep only the valid entries

nuvT_3 = nuvT_2[mask_1d]
f_ophit_PE_3 = f_ophit_PE_2_sel[mask_1d]
f_ophit_ch_3 = f_ophit_ch_2_sel[mask_1d]
f_ophit_t_3 = f_ophit_t_adj_sel[mask_1d]
dEpromx_3 = dEpromx_sel[mask_1d]
dEpromy_3 = dEpromy_sel[mask_1d]
dEpromz_3 = dEpromz_sel[mask_1d]
dEtpc_3 = dEtpc_sel[mask_1d]
nuvZ_3 = nuvZ_2[mask_1d]

### Para reconstrucción espacial

In [26]:
# Create a boolean mask where dEpromx_f_unique is not -999

mask = (dEpromx_sel != -999) & (dEpromy_sel != -999) & (dEpromz_sel != -999)
mask_1d = ak.to_numpy(mask)

# Apply the mask to both the image and dEpromx_f_unique to keep only the valid entries

nuvT_3 = nuvT_2[mask_1d]
f_ophit_PE_3 = f_ophit_PE_2_sel[mask_1d]
f_ophit_ch_3 = f_ophit_ch_2_sel[mask_1d]
f_ophit_t_3 = f_ophit_t_adj_sel[mask_1d]
dEpromx_3 = dEpromx_sel[mask_1d]
dEpromy_3 = dEpromy_sel[mask_1d]
dEpromz_3 = dEpromz_sel[mask_1d]
dEtpc_3 = dEtpc_sel[mask_1d]
nuvZ_3 = nuvZ_2[mask_1d]

In [19]:
print(len(nuvT_2))

101189


In [27]:
print(len(nuvT_3))

100791


In [28]:
del nuvT_2, f_ophit_PE_2_sel, f_ophit_ch_2_sel, f_ophit_t_adj_sel, nuvZ_2

In [ ]:
# Diccionario con los arrays
data = {
    "nuvT": nuvT_3,
    "f_ophit_PE": f_ophit_PE_3,
    "f_ophit_ch": f_ophit_ch_3,
    "f_ophit_t": f_ophit_t_3,
    "dEpromx": dEpromx_3,
    "dEpromy": dEpromy_3,
    "dEpromz": dEpromz_3,
    "nuvZ": nuvZ_3,
}

# Convertir a Arrow Table y guardar como Parquet
table = ak.to_arrow_table(data)
pq.write_table(table, "/exp/sbnd/app/users/svidales/AI_nuvT/v1305_tpcselection_preproc_springval_allevents_50energylimit.parquet")

: 

In [37]:
print(len(f_ophit_PE_3))

75983
